In [1]:
from sqlalchemy.orm import Session
from repository.LectureRepository import LectureRepository
from config.database.db_config import SessionLocal

db: Session = SessionLocal()

lectureRepository = LectureRepository(db)
lectures = lectureRepository.get_all_lectures()

In [2]:
from langchain_core.documents import Document

documents = []

for lecture in lectures:
    doc = Document(
        page_content=lecture.title + ',' + lecture.instructor + ',' + ','.join(lecture.goals + lecture.target + lecture.section_names),
        metadata={
            'id': lecture.id,
            'title': lecture.title,
            'instructor': lecture.instructor,
            'image_url': lecture.image_url,
            'lecture_url': lecture.lecture_url,
            'price': lecture.price
        }
    )
    documents.append(doc)

In [3]:
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')

In [4]:
import os
from pinecone import Pinecone

pinecone_api_key = os.environ.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [5]:
from langchain_pinecone import PineconeVectorStore

index_name = 'inflearn-lectures'

database = PineconeVectorStore.from_documents(documents, embedding, index_name=index_name)


In [8]:
import warnings
from langchain import hub

warnings.filterwarnings("ignore", category=UserWarning, module='langchain')

prompt = hub.pull('rlm/rag-prompt')